# All-Pairs Shortest Path (APSP)

The **All-Pairs Shortest Path (APSP)** problem consists of finding the shortest path between every possible pair of nodes $(u, v)$ within a graph $G = (V, E)$.

---

Based on theoretical analysis and the generator's characteristics, we decided to adopt a hybrid approach:

**If `negative_values = False`, use Dijkstra:**
    It is the fastest and most stable algorithm. It is executed $V$ times (once for each source node). Although A* is superior to Dijkstra for finding the path between *two specific nodes*, we decided to use Dijkstra because A* is inefficient for the *all pairs* problem. A* would need to be run for every source-destination pair ($V^2$ runs) and compute heuristics each time. Dijkstra solves all distances from a single source in one run ($V$ runs total), making it much more efficient for APSP.

**If `negative_values = True`, use Bellman-Ford:**
    Despite the higher computational complexity, it is a mandatory choice to ensure correctness in the presence of negative edges and to detect potential negative cycles generated by the `negative_values`.

---

Below the list of libraries required for running the algorithms is presented.

In [1]:
!pip install igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 54.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import igraph as ig
from igraph import Graph
from itertools import product
from tqdm import tqdm
import time

## Problem Generator

The provided function `create_problem` is designed to generate graph instances on which to test the algorithms.

It supports customization of `density`, `negative_values`, `noise_level` and `seed`:
* **`density`:** Controls how much the graph is full. A density of `1.0` creates a complete graph, while low values simulate sparse graphs;
* **`negative_values`:** If set to `True`, it introduces edges with weights < 0;
* **`noise_level` & `seed`:** Ensure data variability while maintaining the **reproducibility** of the experiments.

In [3]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

## igraph Library

To ensure the correctness and measure the efficiency of the manual Python implementations, a benchmarking strategy is employed using a high-performance reference library `igraph`.

Internally, `igraph` builds the graph using optimized C data structures and relies on highly efficient C implementations of Dijkstra’s and Bellman–Ford. Comparing the Python manual implementation against `igraph` allows us to validate the correctness, by comparing the total cost and reachable pairs.

---

The helper function `igraph_algo` constructs a directed weighted graph from the adjacency matrix `problem` and computes the full all-pairs shortest path distance matrix. If `igraph` detects a negative cycle, the function returns a matrix filled with $-\infty$, ensuring consistency with the expected behavior of the reference algorithm.


[Official documentation of igraph](https://python.igraph.org/en/0.10.5/index.html)


In [4]:
def igraph_algo(problem, algorithm):
    g = Graph.Weighted_Adjacency(problem, mode='directed', attr='weight', loops=False)
    try:
        dist = g.distances(source=None, target=None, weights="weight", mode='out', algorithm=algorithm)
    except ig.InternalError as e:
        if "Negative cycle" in str(e):
            dist = np.full((size, size), -np.inf)
        else:
            raise
    return dist

## Problems with `negative_values = False`

### Dijkstra

* **Principle:** It is a greedy algorithm. It maintains a priority queue of nodes to visit, ordered by estimated minimum distance. At each step, it selects the closest unvisited node and "relaxes" its neighbors (updates their distances if a better path is found).
* **Requirements:** It works **exclusively with non-negative weights** ($w \ge 0$). If an edge is negative, the greedy logic fails.
* **Complexity:** With an optimized binary heap, the complexity for a single source is $O(E + V \log V)$. For APSP (executed $V$ times), it becomes $O(V \cdot E + V^2 \log V)$.

In [ ]:
import heapq


def build_adjacency(graph):
    n = graph.shape[0]
    adj = [[] for _ in range(n)]
    for u in range(n):
        for v in range(n):
            w = graph[u][v]
            if w < np.inf and u != v:
                adj[u].append((v, w))
    return adj


def dijkstra(graph, source, adj):
    n = graph.shape[0]
    dist = np.full(n, np.inf)
    dist[source] = 0.0

    # priority queue: (distance, vertex)
    heap = [(0.0, source)]

    while heap:
        d_u, u = heapq.heappop(heap) # extract node with the smallest distance

        # if we found a shorter path to u before, skip this
        if d_u > dist[u]:
            continue

        for v, w in adj[u]:
            new_dist = d_u + w
            if new_dist < dist[v]:
                dist[v] = new_dist
                heapq.heappush(heap, (new_dist, v))

    return dist



### Testing procedure

The testing procedure runs both the igraph implementation and the manual Python version, measuring execution time, the number of valid shortest paths computed, and the total cost sum. The results consistently show that the manual algorithm matches the reference solution provided by igraph, confirming the correctness of the implementation.

In [ ]:
sizes = [10, 20, 50, 100, 200, 500, 1000]
densities = [0.2, 0.5, 0.8, 1.0]
noise_levels = [0.0, 0.1, 0.5, 0.8]
neg_values = [False]

all_positive_combinations = list(product(sizes, densities, noise_levels, neg_values))

In [ ]:
total = 0
total_matches = 0
total_t_ig = 0
total_t_py = 0

for size, density, noise, neg in tqdm(all_positive_combinations, desc="Processing Dijkstra"):

    # Stats
    pairs_ig, costs_ig = 0, []
    pairs_man, costs_man = 0, []


    problem = create_problem(
        size,
        density=density,
        negative_values=neg,
        noise_level=noise
    )

    # run igraph
    t0 = time.perf_counter()
    dist_igraph = igraph_algo(problem, algorithm="dijkstra")
    for s in range(size):
        for d in range(size):
            if s != d and not np.isinf(dist_igraph[s][d]):
                pairs_ig += 1
                costs_ig.append(dist_igraph[s][d])
    t_ig = time.perf_counter() - t0
    total_t_ig += t_ig

    # our solution
    t0 = time.perf_counter()
    adj = build_adjacency(problem)
    for s in range(size):
        ds = dijkstra(problem, s, adj)
        for d in range(size):
            if s != d and not np.isinf(ds[d]):
                pairs_man += 1
                costs_man.append(ds[d])
    t_py = time.perf_counter() - t0
    total_t_py += t_py

    # VALIDATION LOGIC
    sum_ig = sum(costs_ig)
    sum_man = sum(costs_man)

    if pairs_ig == pairs_man and sum_ig == sum_man:
        total_matches += 1
    else:
        print(f"Results differ (size={size}, density={density}, noise={noise}):")
        print(f"   Pairs: {pairs_ig} (igraph) - {pairs_man} (python)")
        print(f"   Sum of costs: {sum_ig} (igraph) - {sum_man} (python)")

    total += 1

print(f"\nTotal matches = {100 * total_matches / total}%")
print(f"Total execution time iGraph = {total_t_ig:.4f}s")
print(f"Total execution time Python = {total_t_py:.4f}s")

Processing Dijkstra: 100%|██████████| 112/112 [20:52<00:00, 11.19s/it]


Total matches = 100.0%
Total execution time iGraph = 116.0528s
Total execution time Python = 1103.9081s


## Problems with `negative_values = True`

### Bellman-Ford

* **Principle:** It iteratively relaxes **all** edges of the graph $V-1$ times. The idea is that in a graph without negative cycles, the shortest path cannot have more than $V-1$ edges. Unlike Dijkstra's algorithm, Bellman-Ford's repetitive relaxation of all edges ensures that the correct shortest path is found, even if that path involves **negative** edge weights.
* **Negative Cycle Detection:** After $V-1$ iterations, it performs an additional relaxation cycle. If a distance is still updated, it means a negative cycle reachable from the source exists (the path can be improved infinitely).
* **Complexity:** Much more expensive than Dijkstra. For a single source, it is $O(V \cdot E)$. For APSP, it becomes $O(V^2 \cdot E)$, which on dense graphs ($E \approx V^2$) approaches $O(V^4)$.


In [5]:
def bellman_ford(graph, source, weights, rows, cols):
    n = graph.shape[0]
    dist = np.full(n, np.inf)
    dist[source] = 0.0
    predecessor = np.full(n, None, dtype=object)

    for _ in range(n-1):
        prev_dist = dist.copy()

        # compute candidate distances
        candidate_dists = dist[rows] + weights

        # update destinations (cols) with the smallest values found
        np.minimum.at(dist, cols, candidate_dists)

        # early stopping if no changes occurred
        if np.array_equal(dist, prev_dist):
            return dist

    # check for negative cycles
    candidate_dists = dist[rows] + weights
    mask_change = candidate_dists < dist[cols]
    mask_valid = np.isfinite(dist[rows])

    if np.any(mask_change & mask_valid):
        return np.full(size, -np.inf)

    return dist

### Testing procedure

For efficiency reasons, if a negative cycle is detected, we skip the computation for that specific instance and assign a value of $-\infty$ to all pairs. The results show that the manual algorithm almost matches the reference solution provided by igraph. In our opinion, the minor differences in total costs are due to floating-point rounding errors and differences in the internal update order of edges in our Python implementation versus iGraph's C implementation.

In [6]:
sizes = [10, 20, 50, 100, 200, 500, 1000]
densities = [0.2, 0.5, 0.8, 1.0]
noise_levels = [0.0, 0.1, 0.5, 0.8]
neg_values = [True]

all_negative_combinations = list(product(sizes, densities, noise_levels, neg_values))

In [7]:
total = 0
total_matches = 0
total_t_ig = 0
total_t_py = 0
negative_cycle_cnt = 0

for size, density, noise, neg in tqdm(all_negative_combinations, desc="Processing Bellman-Ford"):

    # Stats
    pairs_ig, costs_ig = 0, []
    pairs_man, costs_man = 0, []


    problem = create_problem(
        size,
        density=density,
        negative_values=neg,
        noise_level=noise
    )

    # run igraph
    t0 = time.perf_counter()
    dist_igraph = igraph_algo(problem, algorithm="bellman_ford")
    for s in range(size):
        for d in range(size):
            if s != d and not np.isinf(dist_igraph[s][d]):
                pairs_ig += 1
                costs_ig.append(dist_igraph[s][d])
    t_ig = time.perf_counter() - t0
    total_t_ig += t_ig

    # our solution
    t0 = time.perf_counter()
    # get indices of existing edges and weights
    rows, cols = np.where(np.isfinite(problem))
    weights = problem[rows, cols]

    negative_cycle = False
    for s in range(size):
        ds = bellman_ford(problem, s, weights, rows, cols)
        if np.all(ds == -np.inf):  # found negative cycle
            negative_cycle = True
            break
        for d in range(size):
            if s != d and not np.isinf(ds[d]):
                pairs_man += 1
                costs_man.append(ds[d])
    t_py = time.perf_counter() - t0
    total_t_py += t_py

    if negative_cycle:
        # print(f"\nNegative cycle detected (size={size}, density={density}, noise={noise})")
        negative_cycle_cnt += 1
        if pairs_ig == 0:
            total_matches += 1
        total += 1
        continue

    # VALIDATION LOGIC
    sum_ig = sum(costs_ig)
    sum_man = sum(costs_man)

    if pairs_ig == pairs_man and sum_ig == sum_man:
        total_matches += 1
    else:
        print(f"\nResults differ (size={size}, density={density}, noise={noise}):")
        print(f"   Pairs: {pairs_ig} (igraph) - {pairs_man} (python)")
        print(f"   Sum of costs: {sum_ig} (igraph) - {sum_man} (python)")

    total += 1

print(f"\nTotal matches = {100 * total_matches / total}%")
print(f"Total number of problems that have negative cycle = {negative_cycle_cnt}")
print(f"Total execution time iGraph = {total_t_ig:.4f}s")
print(f"Total execution time Python = {total_t_py:.4f}s")

Processing Bellman-Ford:  26%|██▌       | 29/112 [00:00<00:01, 54.62it/s]


Results differ (size=50, density=0.5, noise=0.1):
   Pairs: 2450 (igraph) - 2450 (python)
   Sum of costs: 1123594.0 (igraph) - 1115873.0 (python)


Processing Bellman-Ford:  41%|████      | 46/112 [00:01<00:01, 45.86it/s]


Results differ (size=100, density=0.2, noise=0.1):
   Pairs: 9900 (igraph) - 9900 (python)
   Sum of costs: 5475278.0 (igraph) - 5439112.0 (python)


Processing Bellman-Ford: 100%|██████████| 112/112 [14:23<00:00,  7.71s/it]


Total matches = 98.21428571428571%
Total number of problems that have negative cycle = 70
Total execution time iGraph = 508.1934s
Total execution time Python = 276.5058s
